In [1]:
import os

from google.cloud import bigquery
import pandas as pd

import log
import peloton_instructor
import peloton_user
import peloton_ride
import peloton_workout

USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']

# configure logging
logger = log.setup_custom_logger('peloton')

# Peloton User
In the section below we create the Peloton user, authenticate with the API, and send user data to a table in BigQuery.

In [2]:

user = peloton_user.PelotonUser(USERNAME, PASSWORD)


INFO - peloton_user - Successfully logged in as xmarcosx


In [3]:
client = bigquery.Client()

table_id = 'peloton.users'

job = client.load_table_from_dataframe(user.to_df(), table_id, job_config=user.get_bigquery_job_config())

job.result()

# Peloton Workouts
In the section below we retrieve all workout ids for the user, get workout metadata, and send to a table in BigQuery.

In [4]:
# retrieve all workout ids
workout_ids = user.get_workout_ids()

# create workout objects
workouts = [peloton_workout.PelotonWorkout(user, workout_id) for workout_id in workout_ids]


INFO - peloton_user - Returning 85 workout ids


In [5]:
table_id = 'peloton.workouts'

payload = pd.concat([workout.to_df() for workout in workouts])

job = client.load_table_from_dataframe(payload, table_id, job_config=workouts[0].get_bigquery_job_config())

job.result()

# Peloton Rides

In [6]:
ride_ids = [workout.ride_id for workout in workouts]
unique_ride_ids = list(dict.fromkeys(ride_ids))

rides = [peloton_ride.PelotonRide(user, ride_id) for ride_id in unique_ride_ids]



In [8]:
table_id = 'peloton.rides'

payload = pd.concat([ride.to_df() for ride in rides])

job = client.load_table_from_dataframe(payload, table_id, job_config=rides[0].get_bigquery_job_config())

job.result()